In [24]:
import torch
import wandb
import math
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import albumentations as A

from sklearn.utils import compute_sample_weight
from sklearn.preprocessing import OneHotEncoder
import skimage as ski

import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision.models import EfficientNet_B0_Weights

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [27]:
# Data Loading Parameters
image_height, image_width = 192, 384

# Augmentation Parameters
brightness = 0.25
contrast = 0.25
saturation = 0.25
hue = 0.25
sharpness_factor = 1.25
zoom_factor = 1.2
degree_factor = 5

# Training Parameters
epochs = 12
batch_size = 16
learning_rate = 5e-4
warmup_epochs = 3
warmup_factor = .1
label_smoothing = .1

EXPERIMENT = "bias"


In [28]:
class CustomDataset(Dataset):
    def __init__(self, file_list, label_list,
                input_transforms,
                color_transforms=None,
                geo_transforms=None,
                geo_trans_vanilla=None,
                processing_level=0):

        # Initialize the list of files and labels
        self.file_list = file_list
        self.label_list = label_list
        self.input_transforms = input_transforms
        self.color_transforms = color_transforms
        self.geo_transforms = geo_transforms
        self.geo_trans_vanilla = geo_trans_vanilla
        self.processing_level = processing_level


    def __len__(self):
        return len(self.file_list)

    def CLAHE_transform(self, image):
            # redice dimension
            image = torch.mean(image, dim=0).numpy()
            # apply CLAHE
            equalized_img = ski.exposure.equalize_adapthist(image, clip_limit=.5, nbins=32) # prevous was clip=.6, nbins=48
            # Use mediean filter to reduce noise
            equalized_img = ski.filters.median(equalized_img, ski.morphology.disk(2))

            return torch.tensor(equalized_img, dtype=torch.float32)

    def __getitem__(self, idx):

        if self.processing_level == 0:
            # LVL 0 - No Background Removal, No Augmentation, Color, No CLAHE
            file = self.file_list[idx]
            input = file / 255
            input = input.astype('float32')
            output = self.label_list[idx]

            input = self.input_transforms(input)
            if self.geo_trans_vanilla is not None:
                input = self.geo_trans_vanilla(input)

            return (input, output)

        if self.processing_level == 1:
            # LVL 1 - No Background Removal, Color Augmentation, Color, No CLAHE
            file = self.file_list[idx]
            input = file / 255
            input = input.astype('float32')
            output = self.label_list[idx]

            if self.color_transforms is not None:
                input = self.color_transforms(image=input.astype('float32'))["image"]

            input = self.input_transforms(input)

            if self.geo_trans_vanilla is not None:
                input = self.geo_trans_vanilla(input)

            return (input, output)

        if self.processing_level == 2:
            # LVL 2 - Background Removal, Color Augmentation, Color ,No CLAHE
            file = self.file_list[idx]
            input = file[:,:,:3].astype('float32') / 255
            mask = file[:,:,3] > 0
            output = self.label_list[idx]

            if self.color_transforms is not None:
                input = self.color_transforms(image=input.astype('float32'))["image"]

            input = self.input_transforms(input)
            mask = self.input_transforms(mask)

            mask = mask.repeat(3, 1, 1)
            input[~mask.squeeze(0)] = 0

            if self.geo_transforms is not None:
                input = self.geo_transforms(input)

            return (input, output)

        if self.processing_level == 3:
            # LVL 3 - Background Removal, Color Augmentation, Greyscale, CLAHE
            file = self.file_list[idx]
            input = file[:,:,:3].astype('float32') / 255
            mask = file[:,:,3] > 0
            output = self.label_list[idx]

            if self.color_transforms is not None:
                input = self.color_transforms(image=input.astype('float32'))["image"]

            input = self.input_transforms(input)
            mask = self.input_transforms(mask)

            input = self.CLAHE_transform(input)

            input[~mask.squeeze(0)] = 0
            input = input.unsqueeze(0)

            if self.geo_transforms is not None:
                input = self.geo_transforms(input)

            return (input, output)

In [29]:
def prepare_data(FOLD, LVL):
    path = "/content/drive/MyDrive/PhD/WingApplication_V3/"
    if LVL < 2:
        file_list = np.load(path + "data/pipeline/{}-images-vanilla.npy".format(EXPERIMENT))
        label_list = np.load(path + "data/pipeline/{}-labels-vanilla.npy".format(EXPERIMENT))
        fold_list = np.load(path + "data/pipeline/{}-fold-vanilla.npy".format(EXPERIMENT))
        path_list = np.load(path +"data/pipeline/{}-path-vanilla.npy".format(EXPERIMENT))

        image_height, image_width = 384, 384

    if LVL >= 2:
        file_list = np.load(path + "data/pipeline/{}-images.npy".format(EXPERIMENT))
        label_list = np.load(path + "data/pipeline/{}-labels.npy".format(EXPERIMENT))
        fold_list = np.load(path + "data/pipeline/{}-fold.npy".format(EXPERIMENT))
        path_list = np.load(path +"data/pipeline/{}-path.npy".format(EXPERIMENT))

        image_height, image_width = 192, 384

    oh_encoder = OneHotEncoder()
    oh_label_list = oh_encoder.fit_transform(label_list.reshape(-1,1)).toarray().astype(np.uint8)

    # Split the dataset into train and test based on fold
    train_file_list = file_list[(fold_list != FOLD) & (fold_list != -1)]
    test_file_list = file_list[(fold_list == FOLD) & (fold_list != -1)]
    ood_test_file_list = file_list[(fold_list == -1)]

    train_label_list = oh_label_list[(fold_list != FOLD) & (fold_list != -1)]
    test_label_list = oh_label_list[(fold_list == FOLD) & (fold_list != -1)]
    ood_test_label_list = oh_label_list[(fold_list == -1)]

    train_path_list = path_list[(fold_list != FOLD) & (fold_list != -1)]
    test_path_list = path_list[(fold_list == FOLD) & (fold_list != -1)]
    ood_test_path_list = path_list[(fold_list == -1)]

    # Compute Sample Weights
    sample_weights = compute_sample_weight('balanced', train_label_list)
    sampler = torch.utils.data.WeightedRandomSampler(weights=sample_weights, num_samples=len(train_file_list), replacement=True)

    # Define the transformations
    input_trans = transforms.Compose([transforms.ToTensor(), transforms.Resize((image_height, image_width))])

    # Updated transforms pipeline
    color_trans = A.Compose([
        # Image Capture Variance
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=.5),
        A.PlanckianJitter(p=.5),
        A.ImageCompression(quality_lower=75, quality_upper=100, p=.25),
        A.Defocus(radius=(1, 3), p=.25),
        A.RandomGamma(gamma_limit=(80, 120), p=.25),
        A.MotionBlur(blur_limit=(3, 3), p=.25),
        A.Downscale(scale_min=0.75, scale_max=1, p=.25),
        # Color Changes
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=.5),
        A.ChannelDropout(channel_drop_range=(1, 1), p=.25),
        # Noise
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), per_channel=True, p=.25),
    ])

    geo_trans_vanilla = transforms.Compose([transforms.v2.RandomZoomOut(fill=0, side_range=(1, 1.1), p=0.75),
                                    transforms.Resize((image_width, image_width)),
                                    transforms.v2.RandomHorizontalFlip(p=0.5),
                                    transforms.v2.RandomRotation(degrees=4),])

    geo_trans = transforms.Compose([transforms.v2.RandomZoomOut(fill=0, side_range=(1, 1.1), p=0.75),
                                    transforms.Resize((image_height, image_width)),
                                    transforms.v2.RandomHorizontalFlip(p=0.5),
                                    transforms.v2.RandomRotation(degrees=4),])

    # Create an instance of the CustomDataset
    train_dataset = CustomDataset(train_file_list, train_label_list, input_trans, color_trans, geo_trans, geo_trans_vanilla, processing_level=LVL)
    test_dataset = CustomDataset(test_file_list, test_label_list, input_trans, processing_level=LVL)
    ood_dataset = CustomDataset(ood_test_file_list, ood_test_label_list,input_trans, processing_level=LVL)

    # Create a DataLoader for the dataset
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=12)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=12)
    ood_dataloader = DataLoader(ood_dataset, batch_size=batch_size, num_workers=12)

    return train_dataloader, test_dataloader, ood_dataloader, oh_encoder, train_path_list, test_path_list, ood_test_path_list


In [30]:
def load_model(preprocessing_level):
    # Adapt model to preprocesseing LVL
    if preprocessing_level == 0:
        model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

         # Calculate the split point for the feature extractor layers
        total_layers = len(list(model.features))  # `features` contains the feature extractor layers
        freeze_layers = total_layers // 2         # Calculate the halfway point

        # Freeze the first 50% of the layers
        for idx, layer in enumerate(model.features):
            if idx < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False

        # Modify the classifier to replace the classification head with a dropout and a single dense layer
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),            # Dropout layer
            nn.Linear(num_features, 4)    # Single output for binary classification
        )

        model = model.to(device)
    if preprocessing_level == 1:
        model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)


         # Calculate the split point for the feature extractor layers
        total_layers = len(list(model.features))  # `features` contains the feature extractor layers
        freeze_layers = total_layers // 2         # Calculate the halfway point

        # Freeze the first 50% of the layers
        for idx, layer in enumerate(model.features):
            if idx < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False

        # Modify the classifier to replace the classification head with a dropout and a single dense layer
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),            # Dropout layer
            nn.Linear(num_features, 4)    # Single output for binary classification
        )

        model = model.to(device)
    if preprocessing_level == 2:
        model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

        # Calculate the split point for the feature extractor layers
        total_layers = len(list(model.features))  # `features` contains the feature extractor layers
        freeze_layers = total_layers // 2         # Calculate the halfway point

        # Freeze the first 50% of the layers
        for idx, layer in enumerate(model.features):
            if idx < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False

        # Modify the classifier to replace the classification head with a dropout and a single dense layer
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),            # Dropout layer
            nn.Linear(num_features, 4)    # Single output for binary classification
        )

        model = model.to(device)

    if preprocessing_level == 3:
        model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)

         # Calculate the split point for the feature extractor layers
        total_layers = len(list(model.features))  # `features` contains the feature extractor layers
        freeze_layers = total_layers // 2         # Calculate the halfway point

        # Freeze the first 50% of the layers
        for idx, layer in enumerate(model.features):
            if idx < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False

        # Modify the first convolution layer to accept a single channel
        # Get the current first convolutional layer
        original_conv = model.features[0][0]

        # Create a new convolutional layer with 1 input channel
        new_conv = nn.Conv2d(
            in_channels=1,                  # Change to 1 channel
            out_channels=original_conv.out_channels,
            kernel_size=original_conv.kernel_size,
            stride=original_conv.stride,
            padding=original_conv.padding,
            bias=original_conv.bias is not None
        )

        # Initialize the new conv layer weights by copying and averaging the weights from the original
        with torch.no_grad():
            new_conv.weight[:] = original_conv.weight.mean(dim=1, keepdim=True)

        # Replace the original convolutional layer with the new single-channel conv layer
        model.features[0][0] = new_conv

        # Modify the classifier to replace the classification head with a dropout and a single dense layer
        num_features = model.classifier[1].in_features
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),            # Dropout layer
            nn.Linear(num_features, 4)    # Single output for binary classification
        )

        model = model.to(device)

    return model

In [31]:
# Define scheduler
def lr_lambda(current_epoch):
    if current_epoch < warmup_epochs:
        # Linear warm-up
        return warmup_factor + (1 - warmup_factor) * (current_epoch / warmup_epochs)
    else:
        # Cosine decay
        progress = (current_epoch - warmup_epochs) / (epochs - warmup_epochs)
        return 0.5 * (1 + math.cos(math.pi * progress))


def train_loop(dataloader, model, epoch, ce_loss, optimizer, scheduler):
    model.train()
    total_loss = 0
    total_acc = 0
    total_batches = len(dataloader)

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = ce_loss(pred, y.argmax(dim=1))
        acc = (pred.argmax(dim=1) == y.argmax(dim=1)).float().mean()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Calculate metrics
        total_loss += loss.item()
        total_acc += acc.item()

    # Average metrics for the epoch
    avg_loss = total_loss / total_batches
    acc = total_acc / total_batches

    print(f"Epoch {epoch}: Loss: {avg_loss:.3f}, Accuracy: {acc:.3f}")

    scheduler.step()

def test_loop(dataloader, model, ce_loss):
    model.eval()
    total_loss = 0
    total_acc = 0
    total_batches = len(dataloader)

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)

            pred = model(X)
            loss = ce_loss(pred, y.argmax(dim=1))
            acc = (pred.argmax(dim=1) == y.argmax(dim=1)).float().mean()

            # Calculate metrics
            total_loss += loss.item()
            total_acc += acc.item()

    # Average metrics for the epoch
    avg_loss = total_loss / total_batches
    acc = total_acc / total_batches

    print(f"Test: Loss: {avg_loss:.3f}, Accuracy: {acc:.3f}")

def evaluation(dataloader, model):
    predictions = []
    targets = []
    for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss
            X = X.to(device)
            y = y.to(device)

            pred = model(X)

            predictions.append(pred.cpu().detach().numpy())
            targets.append(y.cpu().detach().numpy())

    return np.concatenate(predictions), np.concatenate(targets)

In [ ]:
path = "/content/drive/MyDrive/PhD/WingApplication_V3/"
for preprocessing_level in range(4):
    for FOLD in range(5):
        print("PREPROCESSING LEVEL: ", preprocessing_level, "FOLD: ", FOLD)
        train_dataloader, test_dataloader, ood_dataloader, oh_encoder, train_path_list, test_path_list, ood_test_path_list = prepare_data(FOLD, preprocessing_level)
        model = load_model(preprocessing_level)
        # Define loss
        ce_loss = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        # Define optimizer
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

        # TRAIN
        for t in range(epochs):
            train_loop(train_dataloader, model, t, ce_loss, optimizer, scheduler)
            test_loop(test_dataloader, model, ce_loss)
            test_loop(ood_dataloader, model, ce_loss)
            print("----------------------------")

        # TEST & SAVE
        test_df = pd.DataFrame()

        predictions, targets = evaluation(test_dataloader, model)
        predictions_ood, targets_ood = evaluation(ood_dataloader, model)

        test_df["PATH"] = list(test_path_list) + list(ood_test_path_list)
        test_df["PRED"] = list(oh_encoder.inverse_transform(predictions)) + list(oh_encoder.inverse_transform(predictions_ood))
        test_df["TARGET"] = list(oh_encoder.inverse_transform(targets)) + list(oh_encoder.inverse_transform(targets_ood))

        path = "/content/drive/MyDrive/PhD/WingApplication_V3/"
        pd.to_pickle(test_df, path + "results/{}/test_df_{}_{}-{}.pkl".format(EXPERIMENT, EXPERIMENT, FOLD, preprocessing_level))

PREPROCESSING LEVEL:  2 FOLD:  0
Epoch 0: Loss: 1.227, Accuracy: 0.517
Test: Loss: 0.796, Accuracy: 0.942
Test: Loss: 1.694, Accuracy: 0.103
----------------------------
Epoch 1: Loss: 0.669, Accuracy: 0.858
Test: Loss: 0.410, Accuracy: 0.983
Test: Loss: 2.700, Accuracy: 0.013
----------------------------
Epoch 2: Loss: 0.548, Accuracy: 0.907
Test: Loss: 0.391, Accuracy: 0.997
Test: Loss: 2.312, Accuracy: 0.083
----------------------------
Epoch 3: Loss: 0.524, Accuracy: 0.928
Test: Loss: 0.429, Accuracy: 0.978
Test: Loss: 2.969, Accuracy: 0.021
----------------------------
Epoch 4: Loss: 0.499, Accuracy: 0.934
Test: Loss: 0.381, Accuracy: 0.994
Test: Loss: 2.639, Accuracy: 0.030
----------------------------
Epoch 5: Loss: 0.499, Accuracy: 0.937
Test: Loss: 0.498, Accuracy: 0.933
Test: Loss: 3.034, Accuracy: 0.018
----------------------------
Epoch 6: Loss: 0.436, Accuracy: 0.966
Test: Loss: 0.369, Accuracy: 1.000
Test: Loss: 3.056, Accuracy: 0.098
----------------------------
Epoch 7: